In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp plotter/fenster_plotter_utils

In [ ]:
#| export
from PIL import Image, ImageDraw, ImageFont

In [ ]:
#| export
# Funktion zum Zeichnen der Mechanismen
def draw_mechanism(draw, mechanism, start_x, end_x, center_x, center_y, top_y, bottom_y, symbol_size):
    if mechanism == "fest":
        draw.line([(center_x - symbol_size // 2, center_y), (center_x + symbol_size // 2, center_y)], fill="black", width=3)
        draw.line([(center_x, center_y - symbol_size // 2), (center_x, center_y + symbol_size // 2)], fill="black", width=3)
    elif mechanism == "dreh rechts":
        draw.line([(end_x, top_y), (start_x, center_y)], fill="black", width=3)
        draw.line([(start_x, center_y), (end_x, bottom_y)], fill="black", width=3)
    elif mechanism == "dreh links":
        draw.line([(start_x, top_y), (end_x, center_y)], fill="black", width=3)
        draw.line([(end_x, center_y), (start_x, bottom_y)], fill="black", width=3)
    elif mechanism == "kipp":
        draw.line([(start_x, bottom_y), (center_x, top_y)], fill="black", width=3)
        draw.line([(center_x, top_y), (end_x, bottom_y)], fill="black", width=3)
    elif mechanism == "drehkipp rechts":
        draw_mechanism(draw, "dreh rechts", start_x, end_x, center_x, center_y, top_y, bottom_y, symbol_size)
        draw_mechanism(draw, "kipp", start_x, end_x, center_x, center_y, top_y, bottom_y, symbol_size)
    elif mechanism == "drehkipp links":
        draw_mechanism(draw, "dreh links", start_x, end_x, center_x, center_y, top_y, bottom_y, symbol_size)
        draw_mechanism(draw, "kipp", start_x, end_x, center_x, center_y, top_y, bottom_y, symbol_size)

# Funktion zum Zeichnen des Fensters
def draw_window_with_dimensions(width, height, wings, mechanisms, include_shutters=False, scale_factor=1, light_type="none", light_height=0):
    # Skalierte Breite und Höhe
    scaled_width = int(width * scale_factor)
    scaled_height = int(height * scale_factor)
    scaled_light_height = int(light_height * scale_factor) if light_type in ["oberlicht", "unterlicht"] else 0

    # Rollladenkastenhöhe berechnen
    if include_shutters:
        shutter_box_height_mm = 210
        shutter_box_height_pixels = int(shutter_box_height_mm * scale_factor)
    else:
        shutter_box_height_pixels = 0

    # Schriftart laden
    font_size = max(24, min(72, scaled_width // 10))
    try:
        font = ImageFont.truetype("arial.ttf", font_size)
    except IOError:
        font = ImageFont.load_default()

    # Benötigten rechten Rand für Beschriftungen berechnen
    right_space = 0
    text = "210"
    text_bbox = font.getbbox(text)
    text_width = text_bbox[2] - text_bbox[0]
    required_right = 20 + 10 + text_width + 10  # line_offset=20, short_line_length=10
    right_space = max(right_space, required_right)

    text = f"{height}"
    text_bbox = font.getbbox(text)
    text_width = text_bbox[2] - text_bbox[0]
    required_right = 20 + 10 + text_width + 10
    right_space = max(right_space, required_right)

    # Bildgröße festlegen
    left_margin = 75
    img_width = left_margin + scaled_width + right_space
    img_height = 75 + shutter_box_height_pixels + scaled_height + 100  # top margin=75, bottom=100

    # Bild erstellen
    img = Image.new('RGB', (img_width, img_height), color='white')
    draw = ImageDraw.Draw(img)

    # Fensteroberkante festlegen
    window_top = 75 + shutter_box_height_pixels

    # Rollladenkasten zeichnen, falls ausgewählt
    if include_shutters:
        draw.rectangle([(left_margin, 75), (left_margin + scaled_width, 75 + shutter_box_height_pixels)], fill="gray", outline="black")

    # Fensterrahmen zeichnen
    draw.rectangle([(left_margin, window_top), (left_margin + scaled_width, window_top + scaled_height)], outline="black", width=6)

    # Dicken definieren
    frame_thickness = 15
    mullion_width = 5
    sash_frame_width = 10
    transom_width = 5

    # Innere Abmessungen berechnen
    inner_left = left_margin + frame_thickness
    inner_top = window_top + frame_thickness
    inner_right = left_margin + scaled_width - frame_thickness
    inner_bottom = window_top + scaled_height - frame_thickness
    total_inner_width = inner_right - inner_left

    # Positionen für Hauptfenster und Zusatzlichter berechnen
    if light_type == "oberlicht":
        light_top = window_top
        light_bottom = window_top + scaled_light_height
        transom_top = light_bottom
        transom_bottom = transom_top + transom_width
        sash_top = transom_bottom
        sash_bottom = window_top + scaled_height - frame_thickness
    elif light_type == "unterlicht":
        sash_top = window_top + frame_thickness
        sash_bottom = window_top + scaled_height - scaled_light_height - transom_width - frame_thickness
        transom_top = sash_bottom
        transom_bottom = transom_top + transom_width
        light_top = transom_bottom
        light_bottom = window_top + scaled_height
    else:
        sash_top = inner_top
        sash_bottom = inner_bottom
        light_top = None
        light_bottom = None
        transom_top = None
        transom_bottom = None

    # Zusatzlicht zeichnen, falls vorhanden
    if light_type in ["oberlicht", "unterlicht"]:
        # Äußeres Rechteck für Zusatzlicht
        draw.rectangle([(inner_left, light_top), (inner_right, light_bottom)], outline="black", width=3)
        # Glasbereich
        glass_left = inner_left + sash_frame_width
        glass_top = light_top + sash_frame_width
        glass_right = inner_right - sash_frame_width
        glass_bottom = light_bottom - sash_frame_width
        draw.rectangle([(glass_left, glass_top), (glass_right, glass_bottom)], fill=(173, 216, 230), outline="black", width=2)
        # Trennbalken
        draw.rectangle([(inner_left, transom_top), (inner_right, transom_bottom)], fill="black")

    # Flügelbreite berechnen
    if wings > 1:
        sash_width = (total_inner_width - (wings - 1) * mullion_width) / wings
    else:
        sash_width = total_inner_width

    # Flügel und Mechanismen zeichnen
    for i in range(wings):
        start_x = inner_left + i * (sash_width + mullion_width)
        end_x = start_x + sash_width
        # Äußeres Flügelrechteck
        draw.rectangle([(int(start_x), int(sash_top)), (int(end_x), int(sash_bottom))], outline="black", width=3)
        # Inneres Glasrechteck
        glass_left = start_x + sash_frame_width
        glass_top = sash_top + sash_frame_width
        glass_right = end_x - sash_frame_width
        glass_bottom = sash_bottom - sash_frame_width
        draw.rectangle([(int(glass_left), int(glass_top)), (int(glass_right), int(glass_bottom))], fill=(173, 216, 230), outline="black", width=2)
        # Zentrum für Mechanismus berechnen
        center_x = (start_x + end_x) / 2
        center_y = (sash_top + sash_bottom) / 2
        symbol_size = min(sash_width, sash_bottom - sash_top) / 4
        # Mechanismus zeichnen
        mechanism = mechanisms[i].lower()
        draw_mechanism(draw, mechanism, start_x, end_x, center_x, center_y, sash_top, sash_bottom, symbol_size)

    # Mittelpfosten zeichnen, falls mehrere Flügel
    if wings > 1:
        for i in range(wings - 1):
            mullion_start_x = inner_left + i * (sash_width + mullion_width) + sash_width
            mullion_end_x = mullion_start_x + mullion_width
            draw.rectangle([(int(mullion_start_x), int(sash_top)), (int(mullion_end_x), int(sash_bottom))], fill="black")

    # Bemaßungslinien
    line_offset = 20
    short_line_length = 10

    # Horizontale Bemaßung (Breite) unten
    draw.line([(left_margin, window_top + scaled_height + line_offset), (left_margin + scaled_width, window_top + scaled_height + line_offset)], fill="black", width=2)
    draw.line([(left_margin, window_top + scaled_height + line_offset - short_line_length), (left_margin, window_top + scaled_height + line_offset + short_line_length)], fill="black", width=2)
    draw.line([(left_margin + scaled_width, window_top + scaled_height + line_offset - short_line_length), (left_margin + scaled_width, window_top + scaled_height + line_offset + short_line_length)], fill="black", width=2)
    # Breitentext
    text = f"{width}"
    text_bbox = font.getbbox(text)
    text_width = text_bbox[2] - text_bbox[0]
    text_x = (img_width // 2) - (text_width // 2)
    text_y = window_top + scaled_height + line_offset + short_line_length + 5
    draw.text((text_x, text_y), text, fill="black", font=font)

    # Vertikale Bemaßung (Höhe) ausschließlich rechts
    draw.line([(left_margin + scaled_width + line_offset, window_top), (left_margin + scaled_width + line_offset, window_top + scaled_height)], fill="black", width=2)
    draw.line([(left_margin + scaled_width + line_offset - short_line_length, window_top), (left_margin + scaled_width + line_offset + short_line_length, window_top)], fill="black", width=2)
    draw.line([(left_margin + scaled_width + line_offset - short_line_length, window_top + scaled_height), (left_margin + scaled_width + line_offset + short_line_length, window_top + scaled_height)], fill="black", width=2)
    # Höhentext
    text = f"{height} mm"
    text_bbox = font.getbbox(text)
    text_height = text_bbox[3] - text_bbox[1]
    text_x = left_margin + scaled_width + line_offset + short_line_length + 5
    text_y = (window_top + window_top + scaled_height) / 2 - text_height / 2
    draw.text((text_x, text_y), text, fill="black", font=font)

    # Rollladenkastenhöhe Bemaßung, ausschließlich rechts
    if include_shutters:
        # Vertikale Linie
        shutter_dimension_x = left_margin + scaled_width + line_offset  # Offset um Überschneidung zu vermeiden
        draw.line([(shutter_dimension_x, 75), (shutter_dimension_x, 75 + shutter_box_height_pixels)], fill="black", width=2)
        # Kurze horizontale Linien
        draw.line([(shutter_dimension_x - short_line_length, 75), (shutter_dimension_x + short_line_length, 75)], fill="black", width=2)
        draw.line([(shutter_dimension_x - short_line_length, 75 + shutter_box_height_pixels), (shutter_dimension_x + short_line_length, 75 + shutter_box_height_pixels)], fill="black", width=2)
        # Text
        text = "210"
        text_bbox = font.getbbox(text)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        text_x = shutter_dimension_x + short_line_length + 5
        text_y = (75 + (75 + shutter_box_height_pixels)) / 2 - text_height / 2
        draw.text((text_x, text_y), text, fill="black", font=font)

    # Rollladen zeichnen, falls ausgewählt
    if include_shutters:
        shutter_height = 100
        shutter_spacing = 5
        for y in range(window_top, window_top + shutter_height, shutter_spacing):
            draw.line([(left_margin, y), (left_margin + scaled_width, y)], fill="green", width=2)

    # Bild anzeigen
    return img